In [3]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import optuna


In [12]:
df_clientes = pd.read_csv('./datasets/data.csv')

In [13]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [14]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


In [15]:
df_clientes.describe()

,faturamento_mensal,numero_de_funcionarios,idade,inovacao
count,5.000000e+02,500.000000,500.000000,500.00000
mean,1.026716e+06,13.690000,9.254000,4.38800
std,4.206095e+05,3.122403,2.959561,2.90281
min,1.842122e+04,2.000000,0.000000,0.00000
25%,7.632536e+05,12.000000,7.000000,2.00000
50%,1.022957e+06,14.000000,9.000000,4.00000
75%,1.295889e+06,16.000000,11.000000,7.00000
max,2.390677e+06,21.000000,16.000000,9.00000


### Preparar dados para execução do modelo GMM

In [20]:
#selecionar as colunas relevantes 
X = df_clientes.copy()

#separar variaveis por tipo para aplicar o ColumnTransformer
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

In [21]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

In [22]:
# criar o column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ]
)

In [23]:
#transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [24]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]])

### Treinar modelo GMM

In [25]:
# criar função para executar no optuna
def gmm_objective(trial):
    #definindo os hiperparametros a serem ajustados
    n_components = trial.suggest_int('n_components', 3, 10)
    covariance_type = trial.suggest_categorical('covariance_type', ['full', 'tied', 'diag', 'spherical'])

    #instanciar o modelo com os hiperparams
    gmm = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=51)

    #treinar o modelo nos dados
    gmm.fit(X_transformed)

    #calculando o BIC
    bic_gmm = gmm.bic(X_transformed)

    return bic_gmm

In [26]:
#criar estudo optuna
search_space = {'n_components': [3,4,5,6,7,8,9,10],
                'covariance_type': ['full', 'tied', 'diag', 'spherical']}

sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_gmm = optuna.create_study(direction='maximize', sampler=sampler)

[I 2025-06-07 09:49:56,838] A new study created in memory with name: no-name-697c59ce-1e2a-4dec-9b33-bcf01b626f57


In [27]:
estudo_gmm.optimize(gmm_objective, n_trials=32)

[I 2025-06-07 09:50:25,001] Trial 0 finished with value: -177.47638669047626 and parameters: {'n_components': 6, 'covariance_type': 'tied'}. Best is trial 0 with value: -177.47638669047626.
[I 2025-06-07 09:50:25,013] Trial 1 finished with value: -23479.731809069122 and parameters: {'n_components': 9, 'covariance_type': 'diag'}. Best is trial 0 with value: -177.47638669047626.
[I 2025-06-07 09:50:25,027] Trial 2 finished with value: -239.1610329150161 and parameters: {'n_components': 5, 'covariance_type': 'tied'}. Best is trial 0 with value: -177.47638669047626.
[I 2025-06-07 09:50:25,079] Trial 3 finished with value: -16669.2276466825 and parameters: {'n_components': 5, 'covariance_type': 'full'}. Best is trial 0 with value: -177.47638669047626.
[I 2025-06-07 09:50:25,089] Trial 4 finished with value: 1570.0096069322854 and parameters: {'n_components': 3, 'covariance_type': 'diag'}. Best is trial 4 with value: 1570.0096069322854.
[I 2025-06-07 09:50:25,120] Trial 5 finished with value

In [28]:
# melhor configuração obtida pelo optuna
best_params = estudo_gmm.best_params

In [29]:
#Instanciar o modelo com os melhores parametros
best_gmm = GaussianMixture(n_components=best_params['n_components'], covariance_type=best_params['covariance_type'], random_state=51)
best_gmm.fit(X_transformed)

,n_components,3
,covariance_type,'spherical'
,tol,0.001
,reg_covar,1e-06
,max_iter,100
,n_init,1
,init_params,'kmeans'
,weights_init,None
,means_init,None
,precisions_init,None
,random_state,51


In [30]:
# Calcular o bic do melhor modelo
best_bic = best_gmm.bic(X_transformed)

In [32]:
#mostrar melhores parametros e resultados
print(f'Quantidade ideal de components: {best_params['n_components']}')
print(f'Tipo de covariancia: {best_params['covariance_type']}')
print(f'Bic melhor: {best_bic}')

Quantidade ideal de components: 3
Tipo de covariancia: spherical
Bic melhor: 13517.954765258872


### Retornar clusters do melhor modelo GMM

In [33]:
# Executar o predict para retornar os clusters
clusters_gmm = best_gmm.predict(X_transformed)

In [35]:
len(clusters_gmm)

500

In [36]:
#gerar as ´probabilidades de cada registro estar em um cluster
clusters_gmm_probs = best_gmm.predict_proba(X_transformed)

In [37]:
clusters_gmm_probs

array([[9.99979899e-01, 5.05962612e-19, 2.01014149e-05],
       [9.99999996e-01, 2.01413652e-24, 4.29831093e-09],
       [1.07405245e-31, 1.00000000e+00, 6.34448151e-13],
       ...,
       [9.99999998e-01, 8.12525715e-24, 2.26355574e-09],
       [5.34785580e-02, 1.72781225e-08, 9.46521425e-01],
       [7.71293334e-32, 1.00000000e+00, 3.24960959e-13]])

In [38]:
#criar coluna chamada cluster para armazenar o cluster do gmm
df_clientes['cluster'] = clusters_gmm.astype(int)

In [39]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualizar resultados

In [40]:
# Cruzar dados de idade, faturamento mensal e clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='cluster')

In [41]:
# Cruzar dados de inovacao, faturamento mensal e clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='cluster')

In [ ]:
# Cruzar dados de inovacao, idade e clusters
px.scatter(df_clientes, x='inovacao', y='idade', color='cluster')

In [43]:
#salvar modelo e pipeline de transformação
import joblib

joblib.dump(best_gmm, 'modelo_cluster_clientes.pkl')
joblib.dump(preprocessor, 'pipeline_cluster_clientes.pkl')

['pipeline_cluster_clientes.pkl']

In [2]:
import gradio as gr 
import joblib 

modelo = joblib.load('./modelo_cluster_clientes.pkl')
pipeline = joblib.load('./pipeline_cluster_clientes.pkl')

In [4]:
#carregar dados do arquivo upload para o datframe pandas
def clustering(arquivo):
    df_empresas = pd.read_csv(arquivo.name)

    #transformar os dados do dataframe para o modelo
    X_transformed = pipeline.fit_transform(df_empresas)

    #treinar o modelo GMM
    modelo.fit(X_transformed)

    #fazer predições
    cluster_empresas = modelo.predict(X_transformed)

    #criar coluna no dataframe com cluster
    df_empresas['cluster'] = cluster_empresas.astype(int)

    #salvar dataframe par arquivo
    df_empresas.to_csv('./clusters_empresas.csv', index=False)
    return './clusters_empresas.csv'



In [5]:
#executar front end
app = gr.Interface(
    clustering,
    gr.File(file_types=['.csv']),
    'file'
)

app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
